In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack

base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함



## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)
custom_train_test_split.__doc__

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

'df를 입력받아서, userid들을 랜덤해서 1-ratio만큼의 user만 선별하고, 마지막 제출껀만 만들어서 train, test로 리턴 '

((2525956, 23), (2007, 23), (260114, 23))

((2525956, 22), (2525956, 1), (2007, 22), (2007, 1))

In [ ]:
x_valid

In [25]:
x_train

,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,day,...,big_category,problem_num,mid_category,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind
0,0,A060001001,A060000001,2020-03-24 00:17:11,7224,NaN,0,NaN,3,24,...,6,1,1,0.947683,0.222749,1268,0.955022,0.207410,637,train
1,0,A060001002,A060000001,2020-03-24 00:17:14,7225,1.0,1,1.000000,3,24,...,6,2,1,0.947683,0.222749,1268,0.913187,0.281603,3040,train
2,0,A060001003,A060000001,2020-03-24 00:17:22,7225,2.0,2,1.000000,3,24,...,6,3,1,0.947683,0.222749,1268,0.913187,0.281603,3040,train
3,0,A060001004,A060000001,2020-03-24 00:17:29,7225,3.0,3,1.000000,3,24,...,6,4,1,0.947683,0.222749,1268,0.913187,0.281603,3040,train
4,0,A060001005,A060000001,2020-03-24 00:17:36,7225,4.0,4,1.000000,3,24,...,6,5,1,0.947683,0.222749,1268,0.913187,0.281603,3040,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260108,7439,A040197006,A040000197,2020-08-21 07:39:45,2132,7.0,10,0.700000,8,21,...,4,6,197,0.744792,0.460450,143,0.720930,0.471632,279,test
260109,7439,A040130001,A040000130,2020-10-14 23:07:23,8832,8.0,11,0.727273,10,14,...,4,1,130,0.604762,0.499738,127,0.584848,0.499618,193,test
260110,7439,A040130002,A040000130,2020-10-14 23:07:41,8832,8.0,12,0.666667,10,14,...,4,2,130,0.604762,0.499738,127,0.584848,0.499618,193,test
260111,7439,A040130003,A040000130,2020-10-14 23:08:02,8244,9.0,13,0.692308,10,14,...,4,3,130,0.604762,0.499738,127,0.725490,0.476214,111,test


In [3]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)

model_loss = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)

In [4]:
model_loss.fit(
    X=x_train[FEATS],
    y=y_train,
#     early_stopping_rounds=2000,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="logloss",
    verbose=100,
)

preds = model_loss.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[100]	train's binary_logloss: 0.546374	valid's binary_logloss: 0.62917
[200]	train's binary_logloss: 0.54251	valid's binary_logloss: 0.623287
[300]	train's binary_logloss: 0.539063	valid's binary_logloss: 0.619211
[400]	train's binary_logloss: 0.536233	valid's binary_logloss: 0.614343
[500]	train's binary_logloss: 0.533777	valid's binary_logloss: 0.611957
[600]	train's binary_logloss: 0.531641	valid's binary_logloss: 0.609731
[700]	train's binary_logloss: 0.52987	valid's binary_logloss: 0.607706
[800]	train's binary_logloss: 0.527932	valid's binary_logloss: 0.606041
[900]	train's binary_logloss: 0.525736	valid's binary_logloss: 0.602973
[1000]	train's binary_logloss: 0.52384	valid's binary_logloss: 0.600589
[1100]	train's binary_logloss: 0.522147	valid's binary_logloss: 0.599959
[1200]	train's binary_logloss: 0.520663	valid's binary_logloss: 0.598788
[1300]	train's binary_logloss: 0.519211	valid's binary_logloss: 0.597546
[1400]	train's binary_logloss: 0.517914	valid's binary_logloss: 

LGBMClassifier(n_estimators=3200, objective='binary', seed=42, silent=-1)

VALID AUC : 0.7722212283840191 ACC : 0.6925759840558047



In [5]:
model_auc.fit(
    X=x_train[FEATS],
    y=y_train,
    early_stopping_rounds=1000,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="auc",
    verbose=100,
)

preds = model_auc.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[100]	train's auc: 0.757435	train's binary_logloss: 0.546374	valid's auc: 0.70322	valid's binary_logloss: 0.62917
[200]	train's auc: 0.762059	train's binary_logloss: 0.54251	valid's auc: 0.710404	valid's binary_logloss: 0.623287
[300]	train's auc: 0.766217	train's binary_logloss: 0.539063	valid's auc: 0.714973	valid's binary_logloss: 0.619211
[400]	train's auc: 0.769646	train's binary_logloss: 0.536233	valid's auc: 0.721349	valid's binary_logloss: 0.614343
[500]	train's auc: 0.772549	train's binary_logloss: 0.533777	valid's auc: 0.724012	valid's binary_logloss: 0.611957
[600]	train's auc: 0.775027	train's binary_logloss: 0.531641	valid's auc: 0.726944	valid's binary_logloss: 0.609731
[700]	train's auc: 0.777064	train's binary_logloss: 0.52987	valid's auc: 0.729365	valid's binary_logloss: 0.607706
[800]	train's auc: 0.779319	train's binary_logloss: 0.527932	valid's auc: 0.731309	valid's binary_logloss: 0.606041
[900]	train's auc: 0.781924	train's binary_logloss: 0.525736	valid's auc: 0.

LGBMClassifier(n_estimators=3200, objective='binary', seed=42, silent=-1)

VALID AUC : 0.7722212283840191 ACC : 0.6925759840558047



In [6]:
post_slack("done")

In [8]:
lgbm_predict(test, model_auc, FEATS, 'LGBM_Baseline_학습데이터변경.csv')

writing prediction : output/LGBM_Baseline_학습데이터변경.csv


In [9]:
lgbm_predict(test, model_loss, FEATS, 'LGBM_Baseline_학습데이터변경_logloss.csv')

writing prediction : output/LGBM_Baseline_학습데이터변경_logloss.csv


In [11]:
d1 = pd.read_csv('output/LGBM_Baseline_학습데이터변경.csv')

In [12]:
d2 = pd.read_csv('output/LGBM_Baseline_학습데이터변경_logloss.csv')

In [15]:
d3 = pd.read_csv('output/submission_lgbm_add_time_test_tag_std_big_mid_problem_number_3200epoch_valauc_7718.csv')

In [17]:
d1.merge(d2, on='id').merge(d3, on='id')

,id,prediction_x,prediction_y,prediction
0,0,0.489572,0.489572,0.617911
1,1,0.791385,0.791385,0.722200
2,2,0.346894,0.346894,0.411294
3,3,0.962404,0.962404,0.961742
4,4,0.486123,0.486123,0.497752
...,...,...,...,...
739,739,0.298297,0.298297,0.283412
740,740,0.550643,0.550643,0.589468
741,741,0.661735,0.661735,0.617862
742,742,0.715602,0.715602,0.667699


In [23]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    {'objective': 'binary'}, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.227970 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2840
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.628573
[200]	valid_0's binary_logloss: 0.622693
[1000]	valid_0's binary_logloss: 0.603296
[1100]	valid_0's binary_logloss: 0.601964
[1200]	valid_0's binary_logloss: 0.600705
[1300]	valid_0's binary_logloss: 0.598862
[1400]	valid_0's binary_logloss: 0.597412
[1500]	valid_0's binary_logloss: 0.596286
[1600]	valid_0's binary_logloss: 0.594529
[1700]	valid_0's binary_logloss: 0.592822
[1800]	valid_0's binary_logloss: 0.591632
[1900]	valid_0's binary_logloss: 0.59061

In [20]:
lgbm_predict(test, model2, FEATS, 'LGBM_Baseline_학습데이터변경_logloss_기존7627.csv')

writing prediction : output/LGBM_Baseline_학습데이터변경_logloss_기존7627.csv


In [21]:
d4 = pd.read_csv('output/LGBM_Baseline_학습데이터변경_logloss_기존7627.csv')

In [22]:
d1.merge(d2, on='id').merge(d3, on='id').merge(d4, on='id')

,id,prediction_x,prediction_y,prediction_x,prediction_y
0,0,0.489572,0.489572,0.617911,0.617911
1,1,0.791385,0.791385,0.722200,0.722200
2,2,0.346894,0.346894,0.411294,0.411294
3,3,0.962404,0.962404,0.961742,0.961742
4,4,0.486123,0.486123,0.497752,0.497752
...,...,...,...,...,...
739,739,0.298297,0.298297,0.283412,0.283412
740,740,0.550643,0.550643,0.589468,0.589468
741,741,0.661735,0.661735,0.617862,0.617862
742,742,0.715602,0.715602,0.667699,0.667699
